In [ ]:
from fuzzywuzzy import process

def auto_generate_mapping(df1, df2, threshold=70):
    """
    Generate a mapping between columns in df1 and df2 using fuzzy matching.
    
    Parameters:
        df1, df2: pandas DataFrames.
        threshold: minimum matching score (0-100) to accept a pairing.
        
    Returns:
        mapping: dict where keys are normalized column names and values are dictionaries 
                 with keys 'df1' and 'df2' giving the matched column names.
    """
    mapping = {}
    for col in df1.columns:
        best_match, score = process.extractOne(col, df2.columns)
        if score >= threshold:
            # Normalize the key: lower-case and replace spaces with underscores.
            key = col.lower().strip().replace(" ", "_")
            mapping[key] = {"df1": col, "df2": best_match}
    return mapping

# Example usage:
mapping = auto_generate_mapping(cashBookDf, bankStatementDf, threshold=70)
print("Automatically generated mapping:")
print(mapping)


In [ ]:
from itertools import product
import pandas as pd

def extract_transactions_dynamic(row, mapping):
    """
    Given a row containing two records (one from each dataset) and a mapping dictionary,
    dynamically extract the fields.
    
    row: a tuple (or list) where row[0][1] is the record from df1 and row[1][1] is from df2.
    mapping: a dictionary from auto_generate_mapping.
    
    Returns:
        A dictionary with keys prefixed by 'df1_' and 'df2_' for each mapped field.
    """
    rec1 = row[0][1]
    rec2 = row[1][1]
    extracted = {}
    for field, cols in mapping.items():
        extracted[f"df1_{field}"] = rec1[cols["df1"]]
        extracted[f"df2_{field}"] = rec2[cols["df2"]]
    return extracted

# Generate all possible pairs (you might want to use a smarter strategy in production)
pairs = pd.DataFrame(list(product(cashBookDf.iterrows(), bankStatementDf.iterrows())))
# Apply the extraction function to each pair.
pairs = pairs.apply(lambda row: extract_transactions_dynamic(row, mapping), axis=1, result_type="expand")


In [ ]:
from fuzzywuzzy import fuzz

# Iterate over each field from the mapping and compute additional features.
for field in mapping.keys():
    col1 = f"df1_{field}"
    col2 = f"df2_{field}"
    
    # If the field seems to be a date field, compute date difference.
    if "date" in field:
        pairs[col1] = pd.to_datetime(pairs[col1])
        pairs[col2] = pd.to_datetime(pairs[col2])
        pairs[f"{field}_diff"] = abs((pairs[col1] - pairs[col2]).dt.days)
    # If the field is descriptive (text), compute a similarity score.
    elif "desc" in field or "description" in field:
        pairs[f"{field}_similarity"] = pairs.apply(lambda row: fuzz.ratio(str(row[col1]), str(row[col2])), axis=1)
    else:
        # Try converting the columns to numeric and compute the absolute difference.
        try:
            pairs[col1] = pd.to_numeric(pairs[col1])
            pairs[col2] = pd.to_numeric(pairs[col2])
            pairs[f"{field}_diff"] = abs(pairs[col1] - pairs[col2])
        except Exception as e:
            # If conversion fails, you might leave it as is or log the error.
            print(f"Could not compute difference for field {field}: {e}")

# Display a preview of the resulting features.
pairs.head()
